# Ground Plane Fitting and Scan Line Run for 3D LiDAR

Ground Plane Fitting (GPF) and Naive Baseline for 3D LiDAR Segmentation

This notebook implements ground segmentation using the Ground Plane Fitting (GPF) algorithm 
proposed in:

"Fast Segmentation of 3D Point Clouds: A Paradigm on LiDAR Data for Autonomous Vehicle Applications"
by D. Zermas, I. Izzat, and N. Papanikolopoulos, 2017.

The implementation also includes a naive baseline method for comparison, as well as 
basic clustering and visualization tools.

# Imports

In [36]:
# Standard library imports
import os
from pathlib import Path

# Third-party imports
import numpy as np
import yaml
import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree  # Used for spatial clustering

# Local application imports
# Note: Ubuntu users may need to fix libz.so.1 symlink issue (see: https://github.com/heremaps/pptk/issues/3)
import pptk  # To install: pip install pptk

# Hyperparameters

In [37]:
# --- Hyperparameters for Ground Plane Fitting (GPF) ---
NUM_LOWEST_POINTS = 2000           # Number of lowest elevation points used to estimate initial ground seed (LPR)
NUM_ITERATIONS = 5                 # Number of iterations for plane refinement in GPF
SEED_HEIGHT_THRESHOLD = 0.4        # Max height above LPR to consider a point as a ground seed
PLANE_DISTANCE_THRESHOLD = 0.2     # Max distance from plane to classify a point as ground

# --- Parameters for Scan Line Run (SLR) clustering ---
SLR_RUN_DISTANCE_THRESHOLD = 0.2   # Max distance between consecutive points in a scanline to form a run
SLR_MERGE_THRESHOLD = 1.0          # Max distance between runs in adjacent scanlines to be considered the same cluster

# Naive Baseline Method

In [38]:
def naive_ground_extractor(point_cloud: np.ndarray, num_lowest_points: int) -> np.ndarray:
    """
    Naive ground extraction method (baseline).
    
    This simple method selects the points with the lowest Z values 
    and assumes they belong to the ground surface. It does not model 
    the ground plane and is used as a baseline for comparison against 
    more robust algorithms like Ground Plane Fitting (GPF).
    
    Args:
        point_cloud (np.ndarray): N x D array of point cloud data.
        num_lowest_points (int): Number of points with lowest Z values to classify as ground.
    
    Returns:
        ground_indices (np.ndarray): Indices of the selected ground points.
    """

    # Select indices of points with the lowest Z values
    return np.argsort(point_cloud[:, 2])[:num_lowest_points]

# Ground Plane Fitting (GPF)

In [39]:
def extract_initial_seed_indices(
    point_cloud: np.ndarray, 
    num_points: int = 1000, 
    height_threshold: float = 0.4
) -> np.ndarray:
    """
    Extract initial seed points for ground plane estimation (GPF).
    
    Args:
        point_cloud (np.ndarray): N x 3 array of points (x, y, z).
        num_points (int): number of lowest Z points to average as LPR.
        height_threshold (float): threshold to select seeds close to LPR height.
    
    Returns:
        seeds_ids (np.ndarray): indices of points selected as initial seeds.
    """

    # Step 1: Sort the point cloud by Z axis (height)
    sorted_indices = np.argsort(point_cloud[:, 2])  # Get indices sorted by height
    sorted_points = point_cloud[sorted_indices]     # Apply sorting

    # Step 2: Compute LPR (Lowest Point Representative)
    lpr_height = np.mean(sorted_points[:num_points, 2])

    # Step 3: Select point ids that are within threshold distance from LPR
    mask = sorted_points[:, 2] < (lpr_height + height_threshold)
    return sorted_indices[mask]

In [40]:
def estimate_ground_plane(points: np.ndarray) -> "tuple[np.ndarray, float]":
    """
    Estimate the ground plane parameters using Singular Value Decomposition (SVD).
    
    Args:
        points (np.ndarray): N x 3 array (x, y, z) of seed points assumed to be on or near the ground.

    Returns:
        tuple: 
            - normal (np.ndarray): Normal vector (a, b, c) of the estimated ground plane.
            - d (float): Offset term of the estimated plane equation (ax + by + cz + d = 0).
    """
    
    # Step 1: Compute centroid of the seed points
    centroid  = np.mean(points, axis=0)
    centered_points  = points - centroid 

    # Step 2: Compute the covariance matrix of centered points
    covariance_matrix = np.cov(centered_points.T)

    # Step 3: Perform SVD on the covariance matrix to extract principal directions
    _, _, vh = np.linalg.svd(covariance_matrix)

    # Step 4: Normal vector is the direction with smallest variance (last column of V^T)
    normal = vh[-1]

    # Step 5: Compute plane bias using point-normal form: ax + by + cz + d = 0
    d = -np.dot(normal, centroid)

    return (normal, d)

In [41]:
def refine_ground_plane(
    point_cloud: np.ndarray,
    num_points: int = 1000,
    height_threshold: float = 0.4,
    distance_threshold: float = 0.2,
    num_iterations: int = 5
) -> "tuple[np.ndarray, np.ndarray, float]":
    """
    Iteratively refine the ground plane estimation using seed points and distance threshold.
    
    Args:
        point_cloud (np.ndarray): Nx6 array [x, y, z, true_label, pred_label, scanline_id].
        num_points (int): Number of lowest Z points used to compute the initial ground seed height (LPR).
        height_threshold (float): Vertical distance threshold from the LPR used to select initial seed points.
        distance_threshold (float): Max allowed point-to-plane distance for a point to be considered ground.
        num_iterations (int): Number of iterations to refine the plane and ground classification.
    
    Returns:
        tuple: 
            - point_cloud (np.ndarray): Nx6 array [x, y, z, true_label, pred_label, scanline_id], input array with ground points labeled.
            - normal (np.ndarray): Normal vector (a, b, c) of the estimated ground plane.
            - d (float): Offset term of the estimated plane equation (ax + by + cz + d = 0).
    """

    # Step 0: Use only XYZ for plane estimation
    xyz = point_cloud[:, :3]

    # Step 1: Get initial seed points based on lowest Z values
    seed_indices = extract_initial_seed_indices(xyz, num_points, height_threshold)

    for _ in range(num_iterations):
        # Step 2: Estimate ground plane using current seeds
        normal, d = estimate_ground_plane(xyz[seed_indices])

        # Step 3: Compute distances from all points to the estimated plane
        distances = np.abs(np.dot(xyz, normal) + d) / np.linalg.norm(normal)

        # Step 4: Classify as ground if within distance threshold
        is_ground = distances < distance_threshold

        # Step 5: Update seeds with newly classified ground points
        seed_indices = np.where(is_ground)[0]

    # Final ground classification using last iteration's result
    point_cloud[seed_indices, 4] = 9 # Set label = 9 for ground

    return (point_cloud, normal, d)

# Scan Line Run (SLR)

In [42]:
def group_by_scanline(point_cloud: np.ndarray) -> "list[np.ndarray]":
    """
    Group points by their scanline index in a vectorized way.

    Args:
        point_cloud (np.ndarray): N x 6 array [x, y, z, true_label, pred_label, scanline_id].

    Returns:
        list[np.ndarray]: List of arrays. Each array contains the points (N_i x 6)
                          from one scanline, sorted by scanline_id.
    """
    scan_ids = point_cloud[:, 5].astype(int)
    unique_ids = np.unique(scan_ids)

    return [point_cloud[scan_ids == s_id] for s_id in unique_ids]

In [43]:
def find_runs(scanline_points: np.ndarray, distance_threshold: float = 0.5) -> "list[np.ndarray]":
    """
    Identify runs within a single scanline based on distance between consecutive points.

    Args:
        scanline_points (np.ndarray): N x 6 array [x, y, z, true_label, pred_label, scanline_id].
        distance_threshold (float): Distance threshold to consider two points part of the same run.

    Returns:
        list[np.ndarray]: List of arrays where each array contains the points of a run.
    """
    num_points = len(scanline_points)
    runs = []
    current_run_indices = [0]  # start with the index of the first point

    for i in range(1, num_points):
        dist = np.linalg.norm(scanline_points[i, :3] - scanline_points[i - 1, :3])
        if dist < distance_threshold:
            current_run_indices.append(i)
        else:
            runs.append(scanline_points[current_run_indices])
            current_run_indices = [i]

    # append the last run
    runs.append(scanline_points[current_run_indices])

    # Check if first and last points are close (circular case)
    circular_dist = np.linalg.norm(scanline_points[0, :3] - scanline_points[-1, :3])
    # Only merge runs if:
    # - the scanline appears to be circular (first and last points are close), and
    # - there is more than one run (otherwise merging doesn't make sense)
    if circular_dist < distance_threshold and len(runs) > 1:
        # Merge last run with the first
        runs[0] = np.vstack((runs[-1], runs[0]))
        runs.pop()

    return runs

In [44]:
def update_labels(
    runs_current: "list[np.ndarray]",
    runs_above: "list[np.ndarray]",
    label_equivalences: dict,
    merge_threshold: float = 1.0
):
    """
    Update labels of current scanline runs based on proximity to runs from previous scanline using KDTree.

    Args:
        runs_current (list[np.ndarray]): List of N x 6 arrays for current scanline runs.
        runs_above (list[np.ndarray]): List of N x 6 arrays for previous scanline runs.
        label_equivalences (dict): Dictionary of label equivalences.
        merge_threshold (float): Maximum distance to consider connection between runs.
    """
    def resolve_label(label: int) -> int:
        """Find the final label by following the equivalence chain."""
        while label != label_equivalences[label]:
            label = label_equivalences[label]
        return label

    global_label_counter = max(label_equivalences.values()) + 1

    points_above = np.vstack(runs_above)
    tree_above = KDTree(points_above[:, :3])  # use only x, y, z

    for run in runs_current:
        neighbor_labels = set()

        # Check nearest neighbor of each point in current run
        dists, indices = tree_above.query(run[:, :3], k=1)
        for dist, idx in zip(dists[:, 0], indices[:, 0]):
            if dist < merge_threshold:
                neighbor_label = points_above[idx, 4]
                resolved_label = resolve_label(neighbor_label)
                neighbor_labels.add(resolved_label)

        if not neighbor_labels:
            # No close neighbors → assign new label
            while global_label_counter == 9 or global_label_counter in label_equivalences:
                global_label_counter += 1
            run[:, 4] = global_label_counter
            label_equivalences[global_label_counter] = global_label_counter
        else:
            # Inherit the smallest label and unify equivalences
            min_label = min(neighbor_labels)
            run[:, 4] = min_label
            for lbl in neighbor_labels:
                label_equivalences[lbl] = min_label

In [45]:
def extract_clusters(scanlines: "list[np.ndarray]", label_equivalences: dict) -> np.ndarray:
    """
    Apply resolved labels to all points and return a unified point cloud.

    Args:
        scanlines (list[np.ndarray]): List of N x 6 arrays for each scanline.
        label_equivalences (dict): Dictionary of final label equivalences.

    Returns:
        np.ndarray: N x 6 array with updated labels in column 4.
    """
    non_ground_points = np.vstack(scanlines)

    for idx in range(0, len(non_ground_points)):
        non_ground_points[idx][4] = label_equivalences[non_ground_points[idx][4]]

    return non_ground_points

In [46]:
def scan_line_run_clustering(
    point_cloud: np.ndarray, 
    distance_threshold: float = 0.5, 
    merge_threshold: float = 1.0
) -> np.ndarray:
    """
    Perform scan line run clustering on non-ground points (predicted_label == 0).

    This function detects connected components (runs) within scanlines, propagates
    and merges labels across scanlines, and assigns final labels to each point.

    Args:
        point_cloud (np.ndarray): N x 6 array [x, y, z, true_label, predicted_label, scanline_index].
        distance_threshold (float): Distance threshold to consider two points part of the same run.
        merge_threshold (float): Maximum distance to consider connection between runs.

    Returns:
        np.ndarray: Point cloud with updated predicted labels (column 4).
    """
    label_counter = 0
    label_equivalences = {}

    # Filter non-ground points (predicted_label == 0)
    non_ground_mask = point_cloud[:, 4] == 0
    non_ground_points = point_cloud[non_ground_mask]
    ground_points = point_cloud[~non_ground_mask]

    # Group points into scanlines
    scanlines = group_by_scanline(non_ground_points)

    # Initialize clustering with the first scanline
    runs_above = find_runs(scanlines[0], distance_threshold)
    for runs in runs_above:
        label_counter += 1
        if label_counter == 9:  # reserve label 9 for ground
            label_counter += 1
        runs[:, 4] = label_counter
        label_equivalences[label_counter] = label_counter

    scanlines[0] = np.vstack(runs_above)
        
    # Propagate labels through remaining scanlines
    for i in range(1, len(scanlines)):
        runs_current = find_runs(scanlines[i], distance_threshold)
        update_labels(runs_current, runs_above, label_equivalences, merge_threshold)

        scanlines[i] = np.vstack(runs_current)
        runs_above = runs_current

    clustered_points = extract_clusters(scanlines, label_equivalences)
    return np.vstack((clustered_points, ground_points))

# Dataset

In [47]:
class Dataset:
    def __init__(self, data_path: str, split: str = 'train') -> None:
        """
        Initialize dataset loader.

        Args:
            data_path (str or Path): Base path to the SemanticKITTI dataset.
            split (str): Dataset split to use ('train', 'valid', or 'test').
        """
        self.data_path: Path = Path(data_path)
        self.split: str = split
        self.is_test: bool = split == 'test'

        # Paths to YAML config and data folders
        self.yaml_path: Path = self.data_path / 'semantic-kitti.yaml'
        self.velodynes_path: Path = self.data_path / 'data_odometry_velodyne/dataset/sequences'
        self.labels_path: Path = self.data_path / 'data_odometry_labels/dataset/sequences'

        # Load dataset metadata and label mappings
        with open(self.yaml_path, 'r') as file:
            metadata: dict = yaml.safe_load(file)

        self.sequences: list[int] = metadata['split'][split]
        self.learning_map: dict[int, int] = metadata['learning_map']

        # Convert label map to numpy for fast lookup
        max_label: int = max(self.learning_map.keys())
        self.learning_map_np: np.ndarray = np.zeros((max_label + 1,), dtype=np.uint32)
        for raw_label, mapped_label in self.learning_map.items():
            self.learning_map_np[raw_label] = mapped_label

        # Collect all frame paths for selected sequences
        self.frame_paths: list[tuple[str, str]] = self._collect_frame_paths()

    def _collect_frame_paths(self) -> "list[tuple[str, str]]":
        """Collect all (sequence, frame_id) pairs from the dataset split."""
        frame_list = []
        for seq in self.sequences:
            seq_str = f"{int(seq):02d}"
            seq_velo_path = self.velodynes_path/seq_str/'velodyne'
            velo_files = sorted(seq_velo_path.glob('*.bin'))
            for file in velo_files:
                frame_list.append((seq_str, file.stem))
        return frame_list

    def __len__(self) -> int:
        """Return number of samples in the dataset split."""
        return len(self.frame_paths)

    def _compute_scanline_ids(self, point_cloud: np.ndarray, n_scans: int = 64) -> np.ndarray:
        """
        Approximate scanline indices based on point order.

        Args:
            point_cloud (np.ndarray): Nx3 array of 3D points.
            n_scans (int): Number of LiDAR scanlines (e.g., 64 for HDL-64E).

        Returns:
            np.ndarray: Nx1 array with estimated scanline indices (0 to n_scans - 1).
        """
        total_points = point_cloud.shape[0]
        scanline_ids = np.floor(np.linspace(0, n_scans, total_points, endpoint=False)).astype(int)
        return scanline_ids.reshape(-1, 1)

    def __getitem__(self, idx: int) -> "tuple[np.ndarray, dict[str, np.ndarray]]":
        """
        Load a sample from the dataset.

        Args:
            idx (int): Index of the frame to load.

        Returns:
            tuple:
                - point_cloud_with_label (np.ndarray): Nx6 array [x, y, z, true_label, pred_label, scanline_id].
                - item_dict (dict): Contains 'point_cloud', 'label', and 'mask'.
        """

        seq, frame_id = self.frame_paths[idx]

        # Load point cloud (Nx4), drop reflectance
        velodyne_file_path = self.velodynes_path/seq/'velodyne'/f"{frame_id}.bin"
        with open(velodyne_file_path, 'rb') as file:
            point_cloud = np.fromfile(file, dtype=np.float32).reshape(-1, 4)[:, :3]

        # Load and map semantic labels
        if not self.is_test:
            label_file_path = self.labels_path/seq/'labels'/f"{frame_id}.label"
            if label_file_path.exists():
                with open(label_file_path, 'rb') as file:
                    raw_labels = np.fromfile(file, dtype=np.uint32) & 0xFFFF
                labels = self.learning_map_np[raw_labels]
                mask = labels != 0
            else:
                labels = np.zeros(point_cloud.shape[0], dtype=np.uint32)
                mask = np.ones(point_cloud.shape[0], dtype=bool)
        else:
            labels = np.zeros(point_cloud.shape[0], dtype=np.uint32)
            mask = np.ones(point_cloud.shape[0], dtype=bool)

        # Estimate scanline indices
        scanline_ids = self._compute_scanline_ids(point_cloud)

        # Final format: [x, y, z, true_label, predicted_label, scanline_id]
        point_cloud_with_label = np.hstack((
            point_cloud,
            labels.reshape(-1, 1),
            np.zeros((point_cloud.shape[0], 1), dtype=np.float32),
            scanline_ids
        ))

        item_dict = {
            'point_cloud': point_cloud,
            'label': labels,
            'mask': mask
        }

        return point_cloud_with_label, item_dict

# Generate Plane

In [48]:
def generate_plane_points(
    point_cloud: np.ndarray, 
    normal: np.ndarray, 
    d: float, 
    size: float = 30, 
    resolution: float = 0.5
) -> np.ndarray:
    """
    Generate a grid of 3D points lying on a specified plane, and return them with label placeholders.
    The plane is defined by the equation: ax + by + cz + d = 0

    Args:
        point_cloud (np.ndarray): Nx6 array [x, y, z, true_label, pred_label, scanline_id].
        normal (np.ndarray): Plane normal vector [a, b, c].
        d (float): Plane offset in the equation ax + by + cz + d = 0.
        size (float): Half-length of the plane square grid to generate (in meters).
        resolution (float): Spacing between points in the grid.

    Returns:
        np.ndarray: Mx6 array of points with [x, y, z, label1, label2, label3],
                    where the last 3 columns are filled with -1 as placeholders.
    """

    # Compute center of the plane as the centroid of the point cloud
    center = point_cloud[:, :3].mean(axis=0)
    a, b, c = normal

    # Create a mesh grid around the center point in the XY plane
    x_vals = np.arange(center[0] - size, center[0] + size, resolution)
    y_vals = np.arange(center[1] - size, center[1] + size, resolution)
    xx, yy = np.meshgrid(x_vals, y_vals)
    
    # Solve for z using the plane equation: ax + by + cz + d = 0 => z = (-d - ax - by)/c
    zz = (-d - a * xx - b * yy) / c

    # Stack into N x 3 array of [x, y, z]
    xyz = np.stack((xx, yy, zz), axis=-1).reshape(-1, 3)

    # Create label columns filled with -1 as placeholders (e.g., for plane visualization)
    labels = np.full((xyz.shape[0], 3), -1, dtype=np.float32)

    # Concatenate coordinates and labels into a final N x 6 array
    return np.hstack((xyz, labels))

# Visualizer for Point Clouds

In [49]:
class PointCloudVisualizer:
    def __init__(self, point_size: float = 0.03):
        """
        A visualizer class for rendering point clouds using pptk with color-coded semantic labels.

        Args:
            point_size (float): Default size of points in the pptk viewer.
        """
        self.point_size: float = point_size
        self.fixed_colors_rgb: "dict[int, list[float]]" = self._get_fixed_colors_rgb()

    def _get_fixed_colors_rgb(self) -> "dict[int, list[float]]":
        """
        Returns a dictionary mapping label ids to normalized RGB colors.
        The colors follow the SemanticKITTI color convention (converted from BGR to RGB).

        Returns:
            dict: {label: [R, G, B]} with values in [0, 1].
        """
        fixed_colors = {
            -1: [255, 255, 255],  # plane
             0: [0, 0, 0],        # unlabeled
             1: [245, 150, 100],  # car
             2: [245, 230, 100],  # bicycle
             3: [150, 60, 30],    # motorcycle
             4: [180, 30, 80],    # truck
             5: [250, 80, 100],   # other-vehicle
             6: [30, 30, 255],    # person
             7: [200, 40, 255],   # bicyclist
             8: [90, 30, 150],    # motorcyclist
             9: [255, 0, 255],    # road
            10: [255, 150, 255],  # parking
            11: [75, 0, 75],      # sidewalk
            12: [75, 0, 175],     # other-ground
            13: [0, 200, 255],    # building
            14: [50, 120, 255],   # fence
            15: [0, 175, 0],      # vegetation
            16: [0, 60, 135],     # trunk
            17: [80, 240, 150],   # terrain
            18: [150, 240, 255],  # pole
            19: [0, 0, 255],      # traffic-sign
        }
        return {label: [c / 255.0 for c in reversed(rgb)] for label, rgb in fixed_colors.items()}

    def _get_color_map(self, unique_labels: np.ndarray) -> "list[list[float]]":
        """
        Generate a color map for a set of unique labels.
        If a label is not found in the fixed colors, generate a consistent random color.

        Args:
            unique_labels (np.ndarray): Unique label ids to assign colors to.

        Returns:
            list: List of RGB color triplets.
        """
        color_map = []
        for label in unique_labels:
            if label in self.fixed_colors_rgb:
                color_map.append(self.fixed_colors_rgb[label])
            else:
                np.random.seed(label)  # deterministic color per label
                color_map.append(np.random.rand(3))
        return color_map

    def show(
        self,
        point_cloud: np.ndarray,
        show_true_label: bool = False, 
        show_ground: bool = True,
        show_clusters: bool = True,
        show_unlabeled: bool = True,
        show_plane: bool = False,
        point_size: "float | None" = None
    ) -> None:
        """
        Show a point cloud with selected label filters.

        Args:
            point_cloud (np.ndarray): N x 6 array with columns [x, y, z, true_label, pred_label, scanline_index].
            show_true_label (bool): If True, use true labels (column 4); otherwise, use predicted labels (column 5).
            show_ground (bool): Include ground (label == 9).
            show_clusters (bool): Include clusters (label >= 1 and != 9).
            show_unlabeled (bool): Include unlabeled (label == 0).
            show_plane (bool): Include plane (label == -1).
            point_size (float): Override default point size.
        """

        label_col = 3 if show_true_label else 4
        labels = point_cloud[:, label_col]

        # Construct boolean mask for point selection based on label types
        mask = (
            (show_plane & (labels == -1)) |
            (show_unlabeled & (labels == 0)) |
            (show_ground & (labels == 9)) |
            (show_clusters & (labels >= 1) & (labels != 9))
        )

        # Select points and labels based on mask
        xyz = point_cloud[mask, :3]
        visible_labels = labels[mask].astype(int)

        # Map each label to a unique index for pptk visualization
        unique_labels = np.unique(visible_labels)
        label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
        mapped_labels = np.vectorize(label_to_index.get)(visible_labels)

        # Generate color map for the unique labels
        color_map = self._get_color_map(unique_labels)

        # Launch pptk viewer
        viewer = pptk.viewer(xyz, mapped_labels)
        viewer.set(point_size=point_size or self.point_size)
        viewer.color_map(color_map)

# Run Example

## Bin file

In [50]:
dataset = Dataset('../datasets/semantic-kitti-data')
point_cloud, item = dataset[4]

point_cloud, normal, d = refine_ground_plane(point_cloud, 
                                             num_points=NUM_LOWEST_POINTS, 
                                             height_threshold=SEED_HEIGHT_THRESHOLD, 
                                             distance_threshold=PLANE_DISTANCE_THRESHOLD, 
                                             num_iterations=NUM_ITERATIONS)

point_cloud = scan_line_run_clustering(point_cloud, SLR_RUN_DISTANCE_THRESHOLD, SLR_MERGE_THRESHOLD)

plane = generate_plane_points(point_cloud, normal, d)
point_cloud = np.vstack((point_cloud, plane))

In [51]:
visualizer = PointCloudVisualizer()
visualizer.show(point_cloud)

# Result analisys

In [35]:
def calcular_centroides_por_frame(pontos):
    """
    Para um frame, calcula o número de centroides (clusters únicos).
    """
    labels_unicos = np.unique(pontos[:, 4])
    return len(labels_unicos)

def analisar_reducao_dimensionalidade(lista_frames):
    """
    Para uma lista de frames (cada um com shape N x 6), calcula a redução
    da dimensionalidade com base na quantidade de clusters (centroides).

    Gera um gráfico com:
        - Número de clusters por frame
        - Média e desvio padrão
    """
    clusters_por_frame = []

    for i, frame in enumerate(lista_frames):
        num_clusters = calcular_centroides_por_frame(frame)
        clusters_por_frame.append(num_clusters)

    clusters_por_frame = np.array(clusters_por_frame)
    media = np.mean(clusters_por_frame)
    desvio = np.std(clusters_por_frame)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.plot(clusters_por_frame, marker='o', label='Clusters por frame')
    plt.axhline(media, color='green', linestyle='--', label=f'Média = {media:.1f}')
    plt.fill_between(
        range(len(clusters_por_frame)),
        media - desvio,
        media + desvio,
        color='green',
        alpha=0.2,
        label=f'Desvio padrão = {desvio:.1f}'
    )
    plt.title('Redução da Dimensionalidade por Clusterização')
    plt.xlabel('Frame')
    plt.ylabel('Número de Clusters (Centroides)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return clusters_por_frame, media, desvio


lista_frames = list([frame_segmentado_1, frame_segmentado_2, frame_segmentado_3])
clusters_por_frame, media, desvio = analisar_reducao_dimensionalidade(lista_frames)

NameError: name 'frame_segmentado_1' is not defined

In [ ]:
label_id_to_name = {
    -1: "plane",
     0: "unlabeled",
     1: "car",
     2: "bicycle",
     3: "motorcycle",
     4: "truck",
     5: "other-vehicle",
     6: "person",
     7: "bicyclist",
     8: "motorcyclist",
     9: "road",
    10: "parking",
    11: "sidewalk",
    12: "other-ground",
    13: "building",
    14: "fence",
    15: "vegetation",
    16: "trunk",
    17: "terrain",
    18: "pole",
    19: "traffic-sign"
}

def verificar_consistencia_labels(pontos):
    """
    Verifica se todos os pontos de cada cluster (label da posição 4)
    têm o mesmo label verdadeiro (posição 3), e retorna os nomes dos
    labels verdadeiros quando houver inconsistência.
    
    Retorna:
        inconsistentes (dict): {label_algoritmo: [nomes_dos_labels_verdadeiros]}
    """
    inconsistentes = {}

    labels_algoritmo = np.unique(pontos[:, 4])

    for label in labels_algoritmo:
        cluster = pontos[pontos[:, 4] == label]
        labels_verdadeiros = np.unique(cluster[:, 3].astype(int))

        if len(labels_verdadeiros) > 1:
            nomes_labels = [label_id_to_name.get(l, f"desconhecido({l})") for l in labels_verdadeiros]
            inconsistentes[int(label)] = nomes_labels

    return inconsistentes


inconsistencias = verificar_consistencia_labels(frame_segmentado_3)
print(inconsistencias.__len__())
print('\n############### Cluster 9 eh o cluster do ground #################\n')
for cluster_id, nomes in inconsistencias.items():
    print(f"Cluster {cluster_id}: {', '.join(nomes)}")


In [ ]:
main('./pointclouds/1504941055.292141000.pcd')
main('./pointclouds/1504941060.199916000.pcd')

In [ ]:
def run_all_pointclouds(folder_path='./pointclouds'):
    """
    Find and process all .pcd files in the specified folder using the main() function.
    """
    pcd_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.pcd')])

    for filename in pcd_files:
        file_path = os.path.join(folder_path, filename)
        print(f"\nProcessing: {filename}")
        try:
            main(file_path)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Run the batch
# run_all_pointclouds('./pointclouds')